# LAB1 - Model Operationalization & Deployment

In this notebook, we will create the artifacts and scripts to deploy the LSTM model into a webservice on Azure. The artifacts include the model files, and test scripts to validate your model can be used to predict future reliability of the engines based on the present operating characteristics.

In [13]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import json
import shutil
from keras.models import model_from_json
from urllib.request import urlretrieve
import pandas

import h5py

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

In [4]:
# We will store each of these data sets in a local persistance folder

# the model in json format
LSTM_MODEL = './sotckdemo-model/modellstm.json'

# the weights in h5
MODEL_WEIGHTS = './sotckdemo-model/modellstm.h5' 

# and the schema file
SCHEMA_FILE = './sotckdemo-model/service_schema.json'

## Load the test data frame

In [14]:
data = pandas.read_csv("MSFT.csv", index_col='Date')
# Converting the index as date
data.index = pandas.to_datetime(data.index)

In [15]:
data.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2005-01-03,26.80,26.95,26.6490,26.74,65002900.0,0.0,1.0,20.079805,20.192192,19.966669,20.034850,65002900.0
2005-01-04,26.86,27.10,26.6600,26.84,109442100.0,0.0,1.0,20.124760,20.304579,19.974910,20.109775,109442100.0
2005-01-05,26.84,27.10,26.7600,26.78,72463500.0,0.0,1.0,20.109775,20.304579,20.049835,20.064820,72463500.0
2005-01-06,26.86,27.06,26.6399,26.75,76890500.0,0.0,1.0,20.124760,20.274609,19.959850,20.042342,76890500.0
2005-01-07,26.83,26.89,26.6200,26.67,68723300.0,0.0,1.0,20.102282,20.147237,19.944940,19.982403,68723300.0


In [43]:
test_df = data.iloc[-11:]

In [44]:
test_df.head(15)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2017-09-15,74.83,75.390,74.07,75.31,37901927.0,0.0,1.0,74.453158,75.010338,73.696986,74.930741,37901927.0
2017-09-18,75.23,75.970,75.04,75.16,22730355.0,0.0,1.0,74.851144,75.587417,74.662101,74.781496,22730355.0
2017-09-19,75.21,75.710,75.01,75.44,15606870.0,0.0,1.0,74.831245,75.328727,74.632252,75.060086,15606870.0
2017-09-20,75.35,75.550,74.31,74.94,20415084.0,0.0,1.0,74.970540,75.169532,73.935777,74.562604,20415084.0
2017-09-21,75.11,75.240,74.11,74.21,19038998.0,0.0,1.0,74.731748,74.861094,73.736784,73.836281,19038998.0
2017-09-22,73.99,74.510,73.85,74.41,13969937.0,0.0,1.0,73.617388,74.134770,73.478094,74.035273,13969937.0
2017-09-25,74.09,74.250,72.92,73.26,23502422.0,0.0,1.0,73.716885,73.876079,72.552777,72.891065,23502422.0
2017-09-26,73.67,73.810,72.99,73.26,17105469.0,0.0,1.0,73.299000,73.438295,72.622424,72.891065,17105469.0
2017-09-27,73.55,74.170,73.17,73.85,18934048.0,0.0,1.0,73.179604,73.796482,72.801518,73.478094,18934048.0


In [45]:
#normalise data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
cols = ['Open','High','Low','Volume','Close']
#cols = ['Adj. Open','Adj. High','Adj. Low','Adj. Volume','Adj. Close']
df = pandas.DataFrame(scaler.fit_transform(test_df[cols]) , columns=cols, index=test_df.index, dtype="float32") #Normalize
df.head()

,Open,High,Low,Volume,Close
Date,,,,,
2017-09-15,0.712707,0.731481,0.542453,1.000000,0.940367
2017-09-18,0.933702,1.000000,1.000000,0.439913,0.871560
2017-09-19,0.922652,0.879630,0.985849,0.176936,1.000000
2017-09-20,1.000000,0.805556,0.655660,0.354440,0.770642
2017-09-21,0.867403,0.662037,0.561321,0.303639,0.435780


We will need to recreate the feature engineering (creating the sequence features) just as we did in the model building notebook.

We will do this within the webservice so that the service can take the raw  data, and return a scored result predicting the value (label).

When scoreing an unseen observation, the model will not know the true labels. Therefore, we create a score_df without labels.

### Test init() and run() functions to read from the working directory

The web service requires two functions, an init() function that will initialize the web service by loading the model into the service, and a run() function that will engineer the features to match the model call structure, and score that data set. We create the functions in here for testing and debugging.

In [46]:
def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open(LSTM_MODEL, 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(MODEL_WEIGHTS)


In [51]:
def run(score_input): 
    
    amount_of_features = len(score_input.columns)
    data = score_input.as_matrix() #converts to numpy
    seq_len = 10
    result = []
    for index in range(len(data) - seq_len):
        result.append(data[index: index + seq_len])

    result = np.array(result)

    seq_array = np.reshape(result, (result.shape[0], result.shape[1], amount_of_features))  
    
    try:
        prediction = loaded_model.predict_proba(seq_array)
        print(prediction)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))

The webservice test requires an initialize of the webservice, then send the entire scoring data set into the model. We expect to get 1  prediction for each input in the scoring data set.

In [52]:
init()

pred=run(df)
print(pred)

[[0.08342251]]
[[0.08342251181602478]]


## Persist model assets

Next we persist the assets we have created for use in operationalization. First we need to define the schema so the webservice knows what the payload data will look like as it comes in.

In [53]:
# define the input data frame
inputs = {"score_input": SampleDefinition(DataTypes.PANDAS, df)}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath=SCHEMA_FILE)

# save the schema file for deployment
out = json.dumps(json_schema)
with open(SCHEMA_FILE, 'w') as f:
    f.write(out)

The conda dependencies are defined in this webservices_conda.yaml file. This will be used to tell the webservice server which python packages are required to run this web service

In [63]:
%%writefile ./sotckdemo-model/webservices_conda.yaml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

name: project_environment
channels:
- conda-forge
- defaults
dependencies:
  - python=3.5.2
  - pip:
    - azure-common==1.1.8
    - azure-storage==0.36.0
    - numpy==1.14.0 
    - sklearn
    - keras
    - tensorflow
    - h5py

Overwriting ./sotckdemo-model/webservices_conda.yaml


The lstmscore.py file is python code defining the web service operation. It includes both the init() and run() functions defined earlier imports the required libraries. These should be nearly identical to the previous defined versions.

In [64]:
%%writefile ./sotckdemo-model/lstmscore.py

# import the libraries
import keras
import tensorflow
import json
import shutil
import numpy as np


def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open('modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")

def run(score_input): 
    
    amount_of_features = len(score_input.columns)
    data = score_input.as_matrix() #converts to numpy
    seq_len = 10
    result = []
    for index in range(len(data) - seq_len):
        result.append(data[index: index + seq_len])

    result = np.array(result)

    seq_array = np.reshape(result, (result.shape[0], result.shape[1], amount_of_features))  
    
    try:
        prediction = loaded_model.predict_proba(seq_array)
        print(prediction)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))
    
if __name__ == "__main__":
    init()
    run("{\"score_df\": [{\"Close\": 0.9403669834136963, \"High\": 0.7314814925193787, \"Open\": 0.7127071619033813, \"Volume\": 1.0, \"Low\": 0.5424528121948242}]}")

Overwriting ./sotckdemo-model/lstmscore.py


## Packaging

To move the model artifacts around, we'll zip them into one file. We can then retreive this file from the persistance shared folder on your DSVM.

https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

In [65]:
# Compress the operationalization assets for easy blob storage transfer
!ls ./sotckdemo-model/

MODEL_O16N = shutil.make_archive('LSTM_o16n', 'zip', "./sotckdemo-model/")

lstmscore.py  modellstm.json	   webservices_conda.yaml
modellstm.h5  service_schema.json


# Deployment


## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `pdmmodelmanagement`. You only need to do this once for any AMLWB models you would like to deploy. You will need to supply an **ACCOUNT_REGION** and **RESOURCE_GROUP** from your Azure account. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

If you get a `ResourceGroupNotFound` error, you may need to set the correct subscription. This is typically only an issue if your Azure login connects to multiple subscritpions. 

`az account set -s '<subscription name>'`

You can find the `subscription name` or `subscription id` through the (https://portal.azure.com) under the resource group you'd like to use.

## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with an existing model management context (created above) first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

using the same `<ACCOUNT_REGION>` and `<RESOURCE_GROUP>` in the previous section. Then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`

## Deploy a web service 

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (at least `lstmscore.py`, `modellstm.json`, `modellstm.h5`, `service_schema.json` and `webservices_conda.yaml`). If not, in the AML CLI window, change to the directory where the zip file was unpacked. 

The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

`
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
`

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice with the `SERVICE_ID` = `lstmwebservice`. The `SERVICE_ID` must be all lowercase, with no spaces. This command should work with your account and the deployment artifacts created in this notebook.

`
az ml service create realtime -f lstmscore.py -r python -m modellstm.json -m modellstm.h5 -s service_schema.json -c webservices_conda.yaml --cpu 0.1 -n lstmwebservice
`

This command will take some time to execute. 

## Test your deployment.

Once complete, the `az ml service create` command returns sample usage commands to test the service for both PowerShell and the cmd prompt. You can copy and paste this command into the CLI to test the web service. 

```
First obtain the webservice endpoint with the following command. `az ml service usage realtime -i lstmwebservice`

```
> az ml service usage realtime -i lstmwebservice
Scoring URL:
    http://127.0.0.1:32770/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32770/swagger.json

```
